In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/riyajolly/ques-4/FT Data - data.csv
/kaggle/input/datasets/riyajolly/ques-4/Question 4 - Task.csv


In [3]:
df=pd.read_csv("/kaggle/input/datasets/riyajolly/ques-4/FT Data - data.csv")

In [4]:
#Fixing broken urls
df["trans_url_fixed"]=df["transcription_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)
df["audio_url_fixed"]=df["rec_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)

In [5]:
df.head()

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp,trans_url_fixed,audio_url_fixed
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/639...,https://storage.googleapis.com/upload_goai/639...


In [6]:
model_df=pd.read_csv("/kaggle/input/datasets/riyajolly/ques-4/Question 4 - Task.csv")

In [7]:
model_df.head()

,segment_url_link,Human,Model H,Model i,Model k,Model l,Model m,Model n,Unnamed: 8
0,https://storage.googleapis.com/testing_audio_f...,वही अपना खेती बाड़ी और क्या,वही अपना खेती बाड़ी और क्या,वही अपना खेती बाड़ी और क्या,वही अपना खेती बाड़ी और क्या?,वही अपना खेती बाड़ी और क्या,वही अपना खेतीबाड़ी और क्या,वही अपना खेती बाड़ी और क्या,NaN
1,https://storage.googleapis.com/testing_audio_f...,मौनता का अर्थ क्या होता है,मौनता का अर्थ क्या होता है,मौनता का अर्थ क्या होता है?,मौन तागार थके होतई।,मोनता का अर्थ है क्या होता है,मोन ताका हर थक्या होताहए,मौनता का हर थका होता है,NaN
2,https://storage.googleapis.com/testing_audio_f...,और रक्षाबंधन पे चलो बहनों को,और रक्षाबंधन पे चलो बहनों को,और रक्षाबंधन पे चलो बहनों को --,और रक्षाबंधन पे चलो बहनों को?,और रक्षाबंधन पे चलो बहनों को,और रक्षा बंधन पे चलो बहनों को,और रक्षा बंधन पे चलो बहनों को,NaN
3,https://storage.googleapis.com/testing_audio_f...,एक सिंपल और सादा वे में,एक सिंपल और सादा वे में,एक सिंपल और सादा वे में।,एक सिंपल और सादा वे में?,एक सिंपल और सादावे में,एक सिंपल और सादा वे में,एक सिंपल और सादा वे में,NaN
4,https://storage.googleapis.com/testing_audio_f...,आने वाली चुनौतियों का इंतजार करना,आने वाली चुनौतियों का इंतजार करना,आने वाली चुनौतियों का इंतजार करना।,आने वाली चुनौतियों का इंतजार करना?,आने वाली चुनौतियों का इंतजार करना,आने वाली चुनौतियों का इंतजार करना,आने वाली चुनौतियों का इंतजार करना,NaN


### Overall Idea
We cannot trust the human reference blindly since it may contain errors. Instead, we have to combine all the 5 models and take the best of them.

### Word-level Alignment
We choose the word level alignment since it's the best for WER, captures meaningful errors and compatible with ASR evaluation literature.

### When to trust model oves reference
We will use the confidence rule. if 3 or more models agree on a word and the reference differs, then trust the model otherwise- the reference.

### Lattice-based Transcription Generation
From the lattice, we generate a consensus transcript in which, for each slot, we pick the word with the highest word count.

### Calculating WER
WER = (substitutions + deletions + insertions)/Number of total words in consensus


### Process
1. Tokenize all outputs into word lists.
2. Choose first model as anchor.
3. For each other model:
       a. Align with anchor using DP
       b. Merge aligned words with lattice slots.
4. For each slot:
       a. Store word frequency dictionary.

### Generating Consensus Transcript
consensus=[]                                              
for each lattice_slot:                                    
    if majority exists:                                   
        consensus.append(most_frequent_word)              
    else:                                                 
        consensus.append(reference_word)

### Computing WER for each model
for each model:                                                  
    align model_output with consensus                     
    compute substitutions, deletions and insertions       
    calculate WER